<a href="https://colab.research.google.com/github/hanarayan/DF_PRACTICE/blob/main/Data_Frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

spark


In [ ]:
import pandas as pd
from tabulate import tabulate


df = pd.read_csv('County_Health_Rankings.csv')
df.rename(columns=lambda x: x.strip(), inplace=True)
df['Data Release Year'] = pd.to_numeric(df['Data Release Year'], errors='coerce').fillna(0).astype(int)
df["Raw value"] = pd.to_numeric(df["Raw value"], errors="coerce").fillna(0).astype(float)
df["Confidence Interval Lower Bound"] = pd.to_numeric(df["Confidence Interval Lower Bound"], errors="coerce").fillna(0).astype(float)
df["Confidence Interval Upper Bound"] = pd.to_numeric(df["Confidence Interval Upper Bound"], errors="coerce").fillna(0).astype(float)
df.fillna("NOT AVAILABLE", inplace=True)


print(tabulate(df.head(), headers='keys', tablefmt='grid'))

column_info = pd.DataFrame({'SNo': range(1, len(df.columns) + 1),
                            'Column Name': df.columns})

print(tabulate(column_info, headers='keys', tablefmt='grid'))


#df.head().style.set_properties(**{'border': '1px solid black', 'padding': '10px'})

print(f"Number of records are: {len(df)}")


In [ ]:

county_year_counts = df.groupby(['County', 'Data Release Year']).size().reset_index(name='Record Count')

print(tabulate(county_year_counts, headers='keys', tablefmt='grid'))


In [ ]:
county_year_counts = df.groupby(['County', 'Data Release Year','Measure name']).size().reset_index(name='Record Count')

# Display the result using tabulate
print(tabulate(county_year_counts, headers='keys', tablefmt='grid'))

In [ ]:
result = df.groupby(["Measure name", "Data Release Year"]).size().reset_index(name="Record Count")

result["Mean Count"] = result["Record Count"].mean()  # Global mean of all record counts
result["Average Count"] = result["Record Count"] / result["Record Count"].count()  # Per-row average
result["Sum Count"] = result["Record Count"].sum()  # Total sum of record counts
result["Median Count"] = result["Record Count"].median()  # Median count

print(tabulate(result.head(10), headers="keys", tablefmt="grid"))

In [ ]:
result = df.groupby(["Measure name", "Data Release Year", "County", "State"])["Raw value"].sum().reset_index()

# Sort the result by 'Data Release Year' in descending order, then by 'Raw Value' in descending order
result = result.sort_values(["Data Release Year", "Raw value"], ascending=[False, False])

# Get the top 10 rows
top_10_measures = result.head(10)

print(tabulate(top_10_measures, headers="keys", tablefmt="grid"))


In [ ]:
measures = ["Violent crime rate", "Diabetic", "Unemployment rate"]

filtered_df = df[df["Measure name"].isin(measures)]

result = filtered_df.groupby(["County", "Measure name"])["Raw value"].sum().reset_index()
#print(tabulate(result.head(1000), headers="keys", tablefmt="grid"))

pivot_table = result.pivot(index="County", columns="Measure name", values="Raw value")

pivot_table = pivot_table.fillna(0)

# Ensure all expected columns exist, creating them with 0 values if missing
for measure in measures:
    if measure not in pivot_table.columns:
        pivot_table[measure] = 0

pivot_table["Combined Score"] = (pivot_table["Violent crime rate"] +
                                pivot_table["Diabetic"] +
                                pivot_table["Unemployment rate"])

top_counties = pivot_table.sort_values("Combined Score", ascending=False).head(10)  # Get top 10

print(tabulate(top_counties, headers="keys", tablefmt="grid"))

In [ ]:
less_counties = pivot_table.sort_values("Combined Score", ascending=True).head(10)  # Get less 10
print(tabulate(less_counties, headers="keys", tablefmt="grid"))

In [ ]:

correlation_matrix = df[["Raw value", "Confidence Interval Lower Bound", "Confidence Interval Upper Bound"]].corr()
print(tabulate(correlation_matrix, headers='keys', tablefmt='grid'))
